# Building Model
One of the major changes in tensorflow 2 from tensorfllow 1 is eagar execution. Inputs are processed more python-likely. Inputs are processed in the model when they are called. 

All we have to do is two things. The first one is to define the structure of the model. The second one is to define forward computation when they are called. These can be easily done with keras API

## 1. Defining the structure of the model
We can extend keras.Model class to define a model. Structure of structure of the Model can be specified within __init__ method like below


In [15]:
import tensorflow as tf
from tensorflow import keras


In [16]:
class MyModel(keras.Model):
    def __init__(self, input_dim,output_dim):
        super(MyModel,self).__init__()
        self.kernel = self.add_weight("weight", shape=[input_dim,output_dim])
    


> Once we define a Model, we can create an instance of the model

In [9]:
model = MyModel(100,5)

## 2. Defining forward computation
Forward computation defines how inputs are processed in the model. It is defined in the call method of the model. Inputs are forward-propagated by calling the model instance with inptus as an argument. 

In [13]:
class MyModel(keras.Model):
    def __init__(self, input_dim,output_dim):
        super(MyModel,self).__init__()
        self.weight1 = self.add_weight("weight", shape=[input_dim,output_dim])
    
    def call(self,inputs):
        out = tf.matmul(inputs,self.weight1)
        return out

In [14]:
model = MyModel(100,5)
inp = tf.random.normal(shape=(32,100))
out = model(inp)

## 3. Composing layers
Models can have many layers, such as fully-connected layers, dropout layers, normalization layers and etcs. To stack layers within a model and we can freely use layers provided by keras API.

In [32]:
class ComposedModel(keras.Model):
    def __init__(self, output_dim,n_class,dropout_rate,n_layers):
        super(ComposedModel,self).__init__()
        layers = []
        for i in range(n_layers-1):
            layers.append(keras.layers.Dense(output_dim)) #fully_connected layers
            layers.append(keras.layers.Dropout(dropout_rate)) #dropout
            layers.append(keras.layers.LayerNormalization()) #normalization
        layers.append(keras.layers.Dense(n_class))
        self.nets = keras.Sequential(layers) #inputs are sequentially processed 
    
    def call(self,inputs):
        out = self.nets(inputs)
        return out

In [36]:
model = ComposedModel(50,5,0.2,3)
inp = tf.random.normal(shape=(32,100))
out = model(inp)

## 4. Building custom layers
We can easily build custom layers by extending keras.layers.Layer class.

Below is an example of custom fully-connected layers. We can also determine how to initialize weights.

In [30]:
class FullyConnected(keras.layers.Layer):
    def __init__(self, input_dim, output_dim, init_std=0.02,use_bias=True):
        super(FullyConnected, self).__init__()
        self.nf = output_dim
        self.nx = input_dim
        self.initializer_range = init_std
        self.use_bias = use_bias
        self.weight = self.add_weight(
            "{}_weight".format(self.name),
            shape=[self.nx, self.nf],
            initializer=keras.initializers.TruncatedNormal(stddev=init_std))
        if self.use_bias:
            self.bias = self.add_weight(
                "{}_bias".format(self.name),
                shape=[1, self.nf],
                initializer=tf.zeros_initializer())

    def call(self, x):
        x = tf.matmul(x, self.weight)
        if self.use_bias:
            x += self.bias
        return x

In [52]:
class CustomModel(keras.Model):
    def __init__(self, input_dim, hidden_dim, n_class, n_layers, init_std=0.02,use_bias=True):
        super(CustomModel,self).__init__()
        self.n_layers = n_layers
        self.custom_layers = []
        for i in range(n_layers):
            if i ==0:
                self.custom_layers.append(FullyConnected(input_dim,hidden_dim))
            elif i < n_layers -1:
                self.custom_layers.append(FullyConnected(hidden_dim,hidden_dim))
            else:
                self.custom_layers.append(FullyConnected(hidden_dim,n_class))
    def call(self,x):
        for i in range(self.n_layers):
            x = self.custom_layers[i](x)
        return x

In [53]:
model = CustomModel(100,50,5,3)
inp = tf.random.normal(shape=(32,100))
out = model(inp)

## 5. Difference of keras.Model and keras.Layers
Intuitively, we can use the Layer class to define inner computation blocks, and use the Model class to define the outer model.

Technical difference of Model and Layers is that Model has built-in training, evaluation, and prediction loops (model.fit(), model.evaluate(), model.predict()). and it exposes saving and serialization APIs, whereas Layers doesn't.

you can check https://www.tensorflow.org/guide/keras/custom_layers_and_models for more details